In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
import os

In [ ]:
!unzip train.zip
!unzip validation.zip

*defining the number of classes or emotions
*defining the size of the image
*the number of images we give our model to train at one time

In [ ]:
num_classes=5
img_rows, img_cols =48,48
batch_size=8

path for the datasets

In [ ]:
train_data_dir='/content/train'
validation_data_dir ='/content/validation'

flipping and scaling for more variations of the same image

In [ ]:
train_datagen= ImageDataGenerator(rescale=1./255,
                                  rotation_range=30, 
                                  shear_range=0.3, 
                                  zoom_range=0.3, 
                                  width_shift_range=0.4, 
                                  height_shift_range=0.4, 
                                  horizontal_flip=True, 
                                  vertical_flip=True)

In [ ]:
validation_datagen=ImageDataGenerator(rescale=1./255)

following is provided for training.

In [ ]:
train_generator=train_datagen.flow_from_directory(train_data_dir,
                                                  color_mode='grayscale',
                                                  target_size=(img_rows,img_cols),
                                                  batch_size=batch_size,
                                                  class_mode='categorical',
                                                  shuffle=True)

Found 24282 images belonging to 5 classes.


In [ ]:
validation_generator=validation_datagen.flow_from_directory(train_data_dir,
                                                  color_mode='grayscale',
                                                  target_size=(img_rows,img_cols),
                                                  batch_size=batch_size,
                                                  class_mode='categorical',
                                                  shuffle=True)

Found 24282 images belonging to 5 classes.


In [ ]:
model = Sequential()

CNN starts here.
BLOCK 1

In [ ]:
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

BLOCK 2
--No of neurons increased

In [ ]:
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

BLOCK 3 neurons furthur increased

In [ ]:
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

BLOCK 4 neurons increased

In [ ]:
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

BLOCK 5 Flattening the layers

In [ ]:
model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

BLOCK 6 Dense layer

In [ ]:
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

BLOCK 7

In [ ]:
model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

In [ ]:
print(model.summary())

In [ ]:
from keras.optimizers import RMSprop, SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

generating and Saving the best model

In [ ]:
checkpoint = ModelCheckpoint('TrainedModel.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

validation not increasing in 3 rounds (patience) then stop training

In [ ]:
earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True)

if model accuracy not increasing for 3 rounds (patience)reduce learning rate. (learn slowly)

In [ ]:
reduce_learning_rate= ReduceLROnPlateau(monitor='val_loss', 
                                        factor=0.2,
                                        patience=3, 
                                        verbose=1, 
                                        min_delta=0.0001)

In [ ]:
callback_list= [earlystop, checkpoint,reduce_learning_rate]

compile the model

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics=['accuracy'])

In [ ]:
nb_training_samples =24282
nb_validation_samples=5937
epochs=25

In [ ]:
history=model.fit_generator(
    train_generator,
    steps_per_epoch = nb_training_samples//batch_size,
    epochs=epochs,
    callbacks=callback_list,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples//batch_size
)